# Example of use of ``fft_electronic_spin_density``

In [1]:
from fft_electronic_spin_density.utils import Density

ModuleNotFoundError: No module named 'fft_electronic_spin_density.utils'

In [ ]:
density = Density(fname_cube_file='../cube_files/Cu2AC4_rho_sz_256.cube')

In [ ]:
rho_sz_tot, rho_sz_abs_tot = density.integrate_cube_file()

In [ ]:
density.FFT()
density.plot_fft_2D(i_kz=0)
density.write_cube_file_fft(fout='fft_rho_sz.cube')